In [1]:
from dotenv import load_dotenv

load_dotenv("../../.env")

True

## Semantic search

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("resources/acmecorp-employee-handbook.pdf")

data = loader.load()

print(data)

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-11-20T23:23:16+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-11-20T23:23:16+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'resources/acmecorp-employee-handbook.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Employee Handbook\nNon-Disclosure Agreement (NDA) Policy\nEmployees must protect confidential information belonging to the company, its clients, and partners.\nThis includes, but is not limited to, product roadmaps, customer data, internal communications,\nproprietary algorithms, financial information, and unreleased features. Confidential information may not\nbe shared with unauthorized individuals inside or outside the organization. These obligations continue\nafter employment ends.\nWorkplace Conduct Policy\nEmployees must maintain a respectful, professional environment

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(data)

print(len(all_splits))

3


Embedding Models: https://docs.langchain.com/oss/python/integrations/text_embedding

In [5]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [6]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [7]:
ids = vector_store.add_documents(documents=all_splits)

In [8]:
results = vector_store.similarity_search(
    "첫번째 년도에 휴가 일수는 며칠 인가요?"
)

print(results[0])

page_content='prohibited. Violations may result in disciplinary action.
Paid Time Off (PTO) Policy
Full■time employees accrue PTO according to the following schedule:  0–1 years of service: 10 days
per year (0.833 days per month)  1–3 years of service: 15 days per year (1.25 days per month)  3+
years of service: 20 days per year (1.67 days per month) PTO may be used for vacation, personal
needs, or illness. Requests should be submitted in advance through the HR system unless related to
an emergency. Employees may carry over up to 5 unused PTO days per calendar year. Extended
absences exceeding 5 consecutive business days require manager approval.
Travel & Expense Policy
Employees may be reimbursed for reasonable and necessary expenses incurred during approved
business travel. This includes transportation, lodging, meals, and incidental expenses within
established limits. Receipts must be submitted within 14 days of travel. First-class travel, personal' metadata={'producer': 'ReportL

## RAG Agent

In [9]:
from langchain.tools import tool

@tool
def search_handbook(query: str) -> str:
    """Search the employee handbook for information"""
    results = vector_store.similarity_search(query)
    return results[0].page_content

In [10]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=[search_handbook],
    system_prompt="당신은 친절한 에이전트 입니다. 답변을 위해 employee handbook을 참고할 수 있습니다."
    )

In [11]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="첫번째 년도에 휴가 일수는 며칠 인가요?")]}
)

In [12]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='첫번째 년도에 휴가 일수는 며칠 인가요?', additional_kwargs={}, response_metadata={}, id='f886dde0-32f8-4ef2-962d-357791db112b'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 286, 'prompt_tokens': 170, 'total_tokens': 456, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D6v8iLuVawlLhu5h7FhDx8eX3cT11', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3c86-a755-74d1-a611-e1fafb214ea9-0', tool_calls=[{'name': 'search_handbook', 'args': {'query': '첫 해 연차 일수'}, 'id': 'call_ZR2z0Axp4dyEgPtVTSlZaS6J', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 170,

In [13]:
print(response["messages"][-1].content)

답: 첫 해(0–1년 근속)에는 연차가 10일입니다. 월 단위로는 약 0.833일씩 적립됩니다.

참고เพิ่มเติม:
- 이월: 매년 최대 5일까지만 이월 가능.
- 5일 연속을 초과하는 휴가의 경우 매니저 승인이 필요.
